# Join entre edc_resultats et mapping_categories
***

Comment faire une jointure en toute sécurité entre ces les tables edc_resultats et mapping_categories ?



**Résultats exploration**
- **edc_resultats**<br>
  
on a 150 cdparametre avec des paramètres en doublon<br>
exemples
|cdparametre|	cdparametresiseeaux_list	|libmajparametre_list	|libminparametre_list	|casparam_list|
|---|---|---|---|---|
|2555|	TL,TLD	|THALLIUM,THALLIUM DISSOUS	|Thallium,Thallium dissous	|7440-28-0	|
|2059|	DI,DTI	|DOSE INDICATIVE,DOSE TOTALE INDICATIVE (UTILISER DI	|Dose totale indicative (utiliser DI),Dose indicative|	None	|
|8585|	CYANO44,CYANB44	|SYNECHOCYSTIS SP,SYNECHOCYSTIS SP (BIOVOLUME)	|Synechocystis sp,Synechocystis sp (biovolume)	|None|
|2860|	IMAZAQU	|IMIZAQUINE,IMAZAQUINE|	Imizaquine,Imazaquine	|81335-37-7	|


on a 11  cdparametresiseeaux avec des paramètres en doublon<br>
| cdparametresiseeaux | cdparametre_list |libmajparametre_list|libminparametre_list|casparam_list     |
|---|---|---|---|---|
| IMAZAQU             | 2860             | IMIZAQUINE,IMAZAQUINE                                                              | Imazaquine,Imizaquine                                                              | 81335-37-7            |     
| SAXIT               | 7551             | SAXITOXINE TOTALE,SAXITOXINES TOTALES                                              | Saxitoxines totales,Saxitoxine totale                                              | 35523-89-8            |    
| PPCBS               | 5602             | PROPOXYCARBAZONE-SODIUM,PROPOXYCARBAZONE                                           | Propoxycarbazone-sodium,Propoxycarbazone                                           | 181274-15-7           |     
| CLTHALS             | 7717             | CHLOROTHALONIL R417888,CHLOROTHALONIL SA                                           | Chlorothalonil SA,Chlorothalonil R417888                                           | 1418095-02-9          |        
| GIARDIA             | 1064             | GIARDIA SPP,AMIBE GIARDIA                                                          | Amibe giardia,Giardia spp                                                          | NULL                  |              
| MCTOT               | 6278             | SOMME DES MICROCYSTINES ANALYSÉES,SOMME DES MICROCYSTINES ANALYSÉES (CALCUL)       | Somme des microcystines analysées (calcul),Somme des microcystines analysées       | NULL                  |             
| MCTOAEL             | 6278             | MICROCYSTINES ANALYSÉES -TEST ELISA,TOTAL DES MICROCYSTINES ANALYSÉES - TEST ELISA | Microcystines analysées -test ELISA,Total des microcystines analysées - test ELISA | NULL                  |              
| URAN                | 1361             | URANIUM EN ΜG/L,URANIUM EN MICROGRAMME PAR LITRE                                   | Uranium en µg/l                                                                    | 7440-61-1             |              
| ANTXA               | 7550             | ANATOXINE A TOTALE,ANATOXINES A TOTALES                                            | Anatoxines A totales,Anatoxine A totale                                            | 64285-06-9           
| DSPHOS              | 7584,8511        | ANION PHOSPHONATE,DISODIUM PHOSPHONATE                                             | anion phosphonate,Disodium phosphonate                                             | 14901-63-4,13708-85-5 
| PFOS                | 6561             | SULFONATE DE PERFLUOROOCTANE,ACIDE SULFONIQUE DE PERFLUOROOCTANE                   | Acide sulfonique de perfluorooctane,Sulfonate de perfluorooctane                   | 45298-90-6,1763-23-1  |             
| CYSPPS              | 7552             | CYLINDROSPERMOPSINE TOTALE,CYLINDROSPERMOPSINES TOTALES                            | Cylindrospermopsine totale,Cylindrospermopsines totales                            | 143545-90-8           |              


**A priori il vaudrait mieux utiliser cdparametresiseeaux pour l'unicité mais cela ne suffit pas**

**Pour catégorsier, à priori `cdparametre`devrait suffire** : que ce soit dissoud, ou avec une certaine concentration cela ne change pas la catégorisation, j'imagine

- **mapping_categories** : Cette table a beaucoup de doublons
  - des lignes qui diffèrent que par le numéro CAS NULL et non null (exemple : NO2	1339	NITRITES (EN NO2)	Nitrites (en NO2)	)
  - des cdparametresiseeaux qui diffèrent pour un même cdparametre
  - des libmajparametre qui diffèrent pour un même cdparametre/cdparametresiseeaux
  - des libminparametre qui diffèrent pour un même cdparametre/cdparametresiseeaux


**Résultat sur la jointure**<br>
Dans l'état de mapping_categories il faut garder les 4 paramètres pour les jointures

--> je recommande de retravailler la table de mapping pour qu'il n'y ai pas de doublon (a minima pour les numéro CAS)<br>
on pourrait se contenter d'une jointure sur cdparametre et cdparametresiseeaux si la table est bien propre<br>

On pourrait avoir une table avec cdparametresiseeaux , cdparametre , casparam et catégorie <br>
pour les labels pour pourrait faire un choix arbitraire qd il y en a plusieurs ou carrèment ne pas les faire apparaitre

# Exploration
***


In [1]:
### Packages + read data
import pandas as pd
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE


pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

### edc_resultats 
***

In [2]:
# Preview edc_resultats
preview = con.sql("SELECT * FROM edc_resultats  LIMIT 2").df()
preview

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,001,00100119085,12DCLE,1161,"DICHLOROÉTHANE-1,2","Dichloroéthane-1,2",None,N,L,"<0,50",µg/L,133,<=3 µg/L,None,0.0,107-06-2,00100125759,2020,2025-02-14,20230811-150005
1,001,00100119085,A2H,1832,ATRAZINE-2-HYDROXY,Atrazine-2-hydroxy,None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.0,2163-68-0,00100125759,2020,2025-02-14,20230811-150005


In [3]:
# Compter le nombre de paramètre unique par cdparametre

query_test = """
    SELECT
      cdparametre  ,
      string_agg( DISTINCT cdparametresiseeaux) AS cdparametresiseeaux_list,
      string_agg( DISTINCT libmajparametre) AS libmajparametre_list,
      string_agg( DISTINCT libminparametre) AS libminparametre_list,
      string_agg( DISTINCT casparam) AS casparam_list,
      COUNT(DISTINCT cdparametresiseeaux) AS nb_cdparametresiseeaux   ,
      COUNT(DISTINCT libmajparametre) AS nb_libmajparametre ,
      COUNT(DISTINCT libminparametre) AS nb_libminparametre ,	
      COUNT(DISTINCT casparam	) AS nb_casparam 
    FROM 
        edc_resultats
    WHERE 
        cdparametre IS NOT NULL
    GROUP BY
        cdparametre  
    HAVING
       nb_cdparametresiseeaux >1 
       OR nb_libmajparametre > 1
       OR nb_libminparametre >1
       OR nb_casparam > 1
"""

df_test = con.sql(query_test).df()
df_test

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametre,cdparametresiseeaux_list,libmajparametre_list,libminparametre_list,casparam_list,nb_cdparametresiseeaux,nb_libmajparametre,nb_libminparametre,nb_casparam
0,1335,"NH4_NAT,NH4","AMMONIUM (EN NH4),AMMONIUM D'ORIGINE NATURELLE","Ammonium d'origine naturelle,Ammonium (en NH4)",14798-03-9,2,2,2,1
1,1386,"NID,NI","NICKEL DISSOUS,NICKEL","Nickel,Nickel dissous",7440-02-0,2,2,2,1
2,1343,"H2S,H2SULF","HYDROGÈNE SULFURÉ,SULFURES TOTAUX (EN H2S)","Sulfures totaux (en H2S),Hydrogène sulfuré",7783-06-4,2,2,2,1
3,1735,"CLITEMG,CLITE_T","CHLORITE EN MG/L,CHLORITES EN CAS DE TRAITEMENT POUVANT EN GÉNÉRER","Chlorites en cas de traitement pouvant en générer,Chlorite en mg/L",14998-27-7,2,2,2,1
4,2058,"MCLR,MCLRD,MCLRB","MICROCYSTINE-LR TOTALE,MICROCYSTINE-LR DANS LA BIOMASSE,MICROCYSTINE-LR DISSOUTE","Microcystine-LR totale,Microcystine-LR dans la biomasse,Microcystine-LR dissoute",101043-37-2,3,3,3,1
...,...,...,...,...,...,...,...,...,...
145,7142,"DIMEP,DIQDBMD","DIQUAT DIBROMIDE,DIMÉPIPÉRATE","Dimépipérate,Diquat dibromide","61432-55-1,85-00-7",2,2,2,2
146,8865,"471811R,R471811",CHLOROTHALONIL R471811,Chlorothalonil R471811,None,2,1,1,0
147,7729,"NOAMTC,MTCNOA",METOLACHLOR NOA 413173,Metolachlor NOA 413173,1418095-19-8,2,1,1,1
148,5602,PPCBS,"PROPOXYCARBAZONE,PROPOXYCARBAZONE-SODIUM","Propoxycarbazone-sodium,Propoxycarbazone",181274-15-7,1,2,2,1


In [4]:
# Compter le nombre de paramètre unique par cdparametresiseeaux

query_test = """
    SELECT
      cdparametresiseeaux  ,
      string_agg( DISTINCT cdparametre) AS cdparametre_list,
      string_agg( DISTINCT libmajparametre) AS libmajparametre_list,
      string_agg( DISTINCT libminparametre) AS libminparametre_list,
      string_agg( DISTINCT casparam) AS casparam_list,
      COUNT(DISTINCT cdparametre) AS nb_cdparametre   ,
      COUNT(DISTINCT libmajparametre) AS nb_libmajparametre ,
      COUNT(DISTINCT libminparametre) AS nb_libminparametre ,	
      COUNT(DISTINCT casparam) AS nb_casparam 
    FROM 
        edc_resultats
    GROUP BY
        cdparametresiseeaux  
    HAVING
       nb_cdparametre >1 
       OR nb_libmajparametre > 1
       OR nb_libminparametre >1
       OR nb_casparam > 1
"""

df_test = con.sql(query_test).df()
df_test

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametresiseeaux,cdparametre_list,libmajparametre_list,libminparametre_list,casparam_list,nb_cdparametre,nb_libmajparametre,nb_libminparametre,nb_casparam
0,PPCBS,5602,"PROPOXYCARBAZONE-SODIUM,PROPOXYCARBAZONE","Propoxycarbazone-sodium,Propoxycarbazone",181274-15-7,1,2,2,1
1,IMAZAQU,2860,"IMIZAQUINE,IMAZAQUINE","Imazaquine,Imizaquine",81335-37-7,1,2,2,1
2,SAXIT,7551,"SAXITOXINE TOTALE,SAXITOXINES TOTALES","Saxitoxines totales,Saxitoxine totale",35523-89-8,1,2,2,1
3,MCTOT,6278,"SOMME DES MICROCYSTINES ANALYSÉES,SOMME DES MICROCYSTINES ANALYSÉES (CALCUL)","Somme des microcystines analysées (calcul),Somme des microcystines analysées",None,1,2,2,0
4,CLTHALS,7717,"CHLOROTHALONIL R417888,CHLOROTHALONIL SA","Chlorothalonil SA,Chlorothalonil R417888",1418095-02-9,1,2,2,1
5,GIARDIA,1064,"GIARDIA SPP,AMIBE GIARDIA","Amibe giardia,Giardia spp",None,1,2,2,0
6,MCTOAEL,6278,"MICROCYSTINES ANALYSÉES -TEST ELISA,TOTAL DES MICROCYSTINES ANALYSÉES - TEST ELISA","Microcystines analysées -test ELISA,Total des microcystines analysées - test ELISA",None,1,2,2,0
7,ANTXA,7550,"ANATOXINE A TOTALE,ANATOXINES A TOTALES","Anatoxine A totale,Anatoxines A totales",64285-06-9,1,2,2,1
8,URAN,1361,"URANIUM EN ΜG/L,URANIUM EN MICROGRAMME PAR LITRE",Uranium en µg/l,7440-61-1,1,2,1,1
9,DSPHOS,"7584,8511","ANION PHOSPHONATE,DISODIUM PHOSPHONATE","Disodium phosphonate,anion phosphonate","14901-63-4,13708-85-5",2,2,2,2


### mapping_categories
***

In [5]:
# Preview mapping_categories
preview = con.sql("SELECT * FROM mapping_categories LIMIT 2").df()
preview

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,PESTOT,6276,TOTAL DES PESTICIDES ANALYSÉS,Total des pesticides analysés,None,pesticides
1,ATRZ,1107,ATRAZINE,Atrazine,1912-24-9,pesticides


In [6]:
query_Test1 = """
SELECT
    cdparametre,
    COUNT(*) AS nb
FROM 
    mapping_categories
GROUP BY
   1
HAVING 
    nb >1
"""
con.sql(query_Test1).show()

┌─────────────┬───────┐
│ cdparametre │  nb   │
│    int32    │ int64 │
├─────────────┼───────┤
│        2860 │     2 │
│        1341 │     2 │
│        1607 │     2 │
│        1656 │     2 │
│        6854 │     2 │
│        6865 │     2 │
│        1376 │     2 │
│        1385 │     3 │
│        1388 │     2 │
│        1389 │     2 │
│          ·  │     · │
│          ·  │     · │
│          ·  │     · │
│        3166 │     2 │
│        3180 │     2 │
│        8587 │     2 │
│        6420 │     2 │
│        1451 │     4 │
│        1584 │     2 │
│        7583 │     2 │
│        1379 │     2 │
│        1361 │     3 │
│        1444 │     2 │
├─────────────┴───────┤
│ 155 rows (20 shown) │
└─────────────────────┘



In [7]:
query_Test2 = """
SELECT
    cdparametresiseeaux,
    COUNT(*) AS nb
FROM 
    mapping_categories
GROUP BY
   1
HAVING 
    nb >1
"""
con.sql(query_Test2).show()

┌─────────────────────┬───────┐
│ cdparametresiseeaux │  nb   │
│       varchar       │ int64 │
├─────────────────────┼───────┤
│ IMAZAQU             │     2 │
│ PPCBS               │     2 │
│ CL                  │     2 │
│ SAXIT               │     2 │
│ CLTHALS             │     2 │
│ AUSN                │     2 │
│ MCTOT               │     2 │
│ MCTOAEL             │     2 │
│ GIARDIA             │     2 │
│ ANTXA               │     2 │
│ URAN                │     2 │
│ MFTC                │     2 │
│ NO3                 │     2 │
│ NO2                 │     2 │
│ PFOS                │     2 │
│ CYANO53             │     2 │
│ CYSPPS              │     2 │
│ DSPHOS              │     2 │
├─────────────────────┴───────┤
│ 18 rows           2 columns │
└─────────────────────────────┘



In [8]:
# Compter le nombre de paramètre unique par cdparametre

query_test = """
    SELECT
      cdparametre  ,
      COUNT(DISTINCT COALESCE(cdparametresiseeaux  ,'NULL')) AS nb_cdparametresiseeaux   ,
      COUNT(DISTINCT COALESCE(libmajparametre,'NULL')) AS nb_libmajparametre ,
      COUNT(DISTINCT COALESCE(libminparametre,'NULL')) AS nb_libminparametre ,	
      COUNT(DISTINCT COALESCE(casparam,'NULL')	) AS nb_casparam 
    FROM 
        mapping_categories
    GROUP BY
        cdparametre  
    HAVING 
        nb_cdparametresiseeaux >  1
        OR nb_libmajparametre > 1 
        OR nb_libminparametre > 1
        OR nb_casparam > 1
"""

df_test = con.sql(query_test).df()
df_test

,cdparametre,nb_cdparametresiseeaux,nb_libmajparametre,nb_libminparametre,nb_casparam
0,1607,2,2,2,1
1,1656,2,2,2,1
2,6854,2,1,1,1
3,1388,2,2,2,1
4,1389,2,2,2,1
...,...,...,...,...,...
150,1584,2,1,1,1
151,5602,1,2,2,1
152,6561,1,2,2,2
153,1337,1,1,1,2


**Regardons par colonne**
***

**cdparametresiseeaux**

In [9]:
# Compter le nombre de paramètre unique par cdparametre

query_cdparametresiseeaux_doublons = """
WITH liste_doublon AS (
    SELECT
      cdparametre  ,
      COUNT(DISTINCT COALESCE(cdparametresiseeaux  ,'NULL')) AS nb_cdparametresiseeaux   ,
    FROM 
        mapping_categories
    GROUP BY
        cdparametre  
    HAVING 
        nb_cdparametresiseeaux >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametre = liste_doublon.cdparametre
ORDER BY 
    mapping_categories.cdparametre
"""

df_cdparametresiseeaux_doublons = con.sql(query_cdparametresiseeaux_doublons).df()
df_cdparametresiseeaux_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,PHE20,0,PH EQUILIBRE CALCULÉ À 20°C,pH Equilibre Calculé à 20°C,None,paramètre organoleptique
1,DSFTN,0,DISULFOTON (UTILISER DSYST),Disulfoton (utiliser DSYST),298-04-4,non classé
2,B,0,BORE µG/L (UTILISER BMG EN MG/L),Bore µg/L (utiliser BMG en mg/L),7440-42-8,non classé
3,ALT,0,ALU.TOT MG/L (UTILISER ALTMICR µG),Alu.tot mg/L (utiliser ALTMICR µg),None,non classé
4,BSIR,1042,BACT. ET SPORES SULFITO-RÉDU./100ML,Bact. et spores sulfito-rédu./100ml,None,microbio
...,...,...,...,...,...,...
332,ODSAVQ,99999,ODEUR SAVEUR (QUALITATIF),Odeur Saveur (qualitatif),None,paramètre organoleptique
333,ODSAV25,99999,ODEUR SAVEUR À 25°C,Odeur Saveur à 25°C,None,paramètre organoleptique
334,AFF_M,99999,AFFICHAGE EN MAIRIE,Affichage en mairie,None,non classé
335,FREQ02,99999,FRÉQUENTATION CUMULÉE,Fréquentation cumulée,None,non classé


In [10]:
# Compter le nombre de paramètre unique par cdparametre en ignorant  cdparametre NULL, 0 et  99999

query_cdparametresiseeaux_doublons = """
WITH liste_doublon AS (
    SELECT
      cdparametre  ,
      COUNT(DISTINCT COALESCE(cdparametresiseeaux  ,'NULL')) AS nb_cdparametresiseeaux   ,
    FROM 
        mapping_categories
    WHERE
        cdparametre NOT IN ('0','99999')
        AND cdparametre IS NOT NULL
    GROUP BY
        cdparametre  
    HAVING 
        nb_cdparametresiseeaux >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametre = liste_doublon.cdparametre
ORDER BY 
    mapping_categories.cdparametre
"""

df_cdparametresiseeaux_doublons = con.sql(query_cdparametresiseeaux_doublons).df()
df_cdparametresiseeaux_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,BSIR,1042,BACT. ET SPORES SULFITO-RÉDU./100ML,Bact. et spores sulfito-rédu./100ml,None,microbio
1,BSIR50,1042,BACT. ET SPORES SULFITO-RÉDU./50ML,Bact. et spores sulfito-rédu./50ml,None,microbio
2,ANAE50,1042,SPORES BACT.ANAÉR.SULFITO-RÉD./50ML,Spores Bact.Anaér.Sulfito-réd./50ml,None,microbio
3,PSA100,1046,PSEUDOMONAS AÉRUGINOSA PAR 100ML,Pseudomonas aéruginosa par 100ml,None,microbio
4,PSA250,1046,PSEUDOMONAS AÉRUGINOSA N/250ML -12H,Pseudomonas aéruginosa n/250ml -12h,None,microbio
...,...,...,...,...,...,...
323,CYANB42,8586,LEPTOLYNGBYA (BIOVOLUME),Leptolyngbya (biovolume),None,microbio
324,CYANO55,8587,CYANOBIUM SP,Cyanobium sp,None,microbio
325,CYANB55,8587,CYANOBIUM SP (BIOVOLUME),Cyanobium sp (biovolume),None,microbio
326,R471811,8865,CHLOROTHALONIL R471811,Chlorothalonil R471811,None,métabolite de pesticide


**libmajparametre**

In [11]:
# Compter le nombre de paramètre unique par cdparametre

query_libmajparametre_doublons = """
WITH liste_doublon AS (
    SELECT
      cdparametre  ,
      COUNT(DISTINCT COALESCE(libmajparametre,'NULL')) AS nb_libmajparametre ,
    FROM 
        mapping_categories
    WHERE
        cdparametre NOT IN ('0','99999')
        AND cdparametre IS NOT NULL
    GROUP BY
        cdparametre  
    HAVING 
        nb_libmajparametre >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametre = liste_doublon.cdparametre
ORDER BY 
    mapping_categories.cdparametre
"""

df_libmajparametre_doublons = con.sql(query_libmajparametre_doublons).df()
df_libmajparametre_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,BSIR,1042,BACT. ET SPORES SULFITO-RÉDU./100ML,Bact. et spores sulfito-rédu./100ml,None,microbio
1,BSIR50,1042,BACT. ET SPORES SULFITO-RÉDU./50ML,Bact. et spores sulfito-rédu./50ml,None,microbio
2,ANAE50,1042,SPORES BACT.ANAÉR.SULFITO-RÉD./50ML,Spores Bact.Anaér.Sulfito-réd./50ml,None,microbio
3,PSA100,1046,PSEUDOMONAS AÉRUGINOSA PAR 100ML,Pseudomonas aéruginosa par 100ml,None,microbio
4,PSA250,1046,PSEUDOMONAS AÉRUGINOSA N/250ML -12H,Pseudomonas aéruginosa n/250ml -12h,None,microbio
...,...,...,...,...,...,...
297,CYANB44,8585,SYNECHOCYSTIS SP (BIOVOLUME),Synechocystis sp (biovolume),None,microbio
298,CYANO42,8586,LEPTOLYNGBYA (CELLULES),Leptolyngbya (cellules),None,microbio
299,CYANB42,8586,LEPTOLYNGBYA (BIOVOLUME),Leptolyngbya (biovolume),None,microbio
300,CYANO55,8587,CYANOBIUM SP,Cyanobium sp,None,microbio


**nb_libminparametre**

In [12]:
# Compter le nombre de paramètre unique par cdparametresiseeaux

query_libmajparametre_doublons = """
WITH liste_doublon AS (
    SELECT
      cdparametresiseeaux  ,
      COUNT(DISTINCT COALESCE(libmajparametre,'NULL')) AS nb_libmajparametre ,
    FROM 
        mapping_categories
    WHERE
        cdparametre NOT IN ('0','99999')
        AND cdparametresiseeaux IS NOT NULL
    GROUP BY
        cdparametresiseeaux  
    HAVING 
        nb_libmajparametre >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametresiseeaux = liste_doublon.cdparametresiseeaux
ORDER BY 
    mapping_categories.cdparametresiseeaux
"""

df_libmajparametre_doublons = con.sql(query_libmajparametre_doublons).df()
df_libmajparametre_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,ANTXA,7550,ANATOXINE A TOTALE,Anatoxine A totale,64285-06-9,microbio
1,ANTXA,7550,ANATOXINES A TOTALES,Anatoxines A totales,64285-06-9,microbio
2,CLTHALS,7717,CHLOROTHALONIL R417888,Chlorothalonil R417888,1418095-02-9,métabolite de pesticide
3,CLTHALS,7717,CHLOROTHALONIL SA,Chlorothalonil SA,1418095-02-9,métabolite de pesticide
4,CYSPPS,7552,CYLINDROSPERMOPSINE TOTALE,Cylindrospermopsine totale,143545-90-8,microbio
5,CYSPPS,7552,CYLINDROSPERMOPSINES TOTALES,Cylindrospermopsines totales,143545-90-8,microbio
6,DSPHOS,7584,DISODIUM PHOSPHONATE,Disodium phosphonate,13708-85-5,non classé
7,DSPHOS,8511,ANION PHOSPHONATE,anion phosphonate,14901-63-4,non classé
8,GIARDIA,1064,GIARDIA SPP,Giardia spp,None,microbio
9,GIARDIA,1064,AMIBE GIARDIA,Amibe giardia,None,microbio


In [13]:
# Compter le nombre de paramètre unique par cdparametre

query_libminparametre_doublons = """
WITH liste_doublon AS (
    SELECT
      cdparametre  ,
      COUNT(DISTINCT COALESCE(libminparametre,'NULL')) AS nb_libminparametre ,	
    FROM 
        mapping_categories
    GROUP BY
        cdparametre  
    HAVING 
        nb_libminparametre >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametre = liste_doublon.cdparametre
ORDER BY 
    mapping_categories.cdparametre
"""

df_libminparametre_doublons = con.sql(query_libminparametre_doublons).df()
df_libminparametre_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,PHE20,0,PH EQUILIBRE CALCULÉ À 20°C,pH Equilibre Calculé à 20°C,None,paramètre organoleptique
1,DSFTN,0,DISULFOTON (UTILISER DSYST),Disulfoton (utiliser DSYST),298-04-4,non classé
2,B,0,BORE µG/L (UTILISER BMG EN MG/L),Bore µg/L (utiliser BMG en mg/L),7440-42-8,non classé
3,ALT,0,ALU.TOT MG/L (UTILISER ALTMICR µG),Alu.tot mg/L (utiliser ALTMICR µg),None,non classé
4,BSIR,1042,BACT. ET SPORES SULFITO-RÉDU./100ML,Bact. et spores sulfito-rédu./100ml,None,microbio
...,...,...,...,...,...,...
306,ODSAVQ,99999,ODEUR SAVEUR (QUALITATIF),Odeur Saveur (qualitatif),None,paramètre organoleptique
307,ODSAV25,99999,ODEUR SAVEUR À 25°C,Odeur Saveur à 25°C,None,paramètre organoleptique
308,AFF_M,99999,AFFICHAGE EN MAIRIE,Affichage en mairie,None,non classé
309,FREQ02,99999,FRÉQUENTATION CUMULÉE,Fréquentation cumulée,None,non classé


In [14]:
# Compter le nombre de paramètre unique par cdparametresiseeaux

query_libminparametre_doublons = """
WITH liste_doublon AS (
    SELECT
      cdparametresiseeaux  ,
      COUNT(DISTINCT COALESCE(libminparametre,'NULL')) AS nb_libminparametre ,	
    FROM 
        mapping_categories
    GROUP BY
        cdparametresiseeaux  
    HAVING 
        nb_libminparametre >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametresiseeaux = liste_doublon.cdparametresiseeaux
ORDER BY 
    mapping_categories.cdparametresiseeaux
"""

df_libminparametre_doublons = con.sql(query_libminparametre_doublons).df()
df_libminparametre_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,ANTXA,7550,ANATOXINE A TOTALE,Anatoxine A totale,64285-06-9,microbio
1,ANTXA,7550,ANATOXINES A TOTALES,Anatoxines A totales,64285-06-9,microbio
2,CLTHALS,7717,CHLOROTHALONIL R417888,Chlorothalonil R417888,1418095-02-9,métabolite de pesticide
3,CLTHALS,7717,CHLOROTHALONIL SA,Chlorothalonil SA,1418095-02-9,métabolite de pesticide
4,CYANO53,<NA>,JAAGINEMA SP,None,None,microbio
5,CYANO53,<NA>,JAAGINEMA SP,Jaaginema sp,None,microbio
6,CYSPPS,7552,CYLINDROSPERMOPSINE TOTALE,Cylindrospermopsine totale,143545-90-8,microbio
7,CYSPPS,7552,CYLINDROSPERMOPSINES TOTALES,Cylindrospermopsines totales,143545-90-8,microbio
8,DSPHOS,7584,DISODIUM PHOSPHONATE,Disodium phosphonate,13708-85-5,non classé
9,DSPHOS,8511,ANION PHOSPHONATE,anion phosphonate,14901-63-4,non classé


**casparam**

In [15]:
# Compter le nombre de paramètre unique par cdparametre

query_casparam_doublons = """
WITH liste_doublon AS (
    SELECT
      cdparametre  ,
      COUNT(DISTINCT COALESCE(casparam,'NULL')	) AS nb_casparam 
    FROM 
        mapping_categories
    GROUP BY
        cdparametre  
    HAVING 
        nb_casparam >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametre = liste_doublon.cdparametre
ORDER BY 
    mapping_categories.cdparametre
"""

df_casparam_doublons = con.sql(query_casparam_doublons).df()
df_casparam_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,PHE20,0,PH EQUILIBRE CALCULÉ À 20°C,pH Equilibre Calculé à 20°C,None,paramètre organoleptique
1,DSFTN,0,DISULFOTON (UTILISER DSYST),Disulfoton (utiliser DSYST),298-04-4,non classé
2,B,0,BORE µG/L (UTILISER BMG EN MG/L),Bore µg/L (utiliser BMG en mg/L),7440-42-8,non classé
3,ALT,0,ALU.TOT MG/L (UTILISER ALTMICR µG),Alu.tot mg/L (utiliser ALTMICR µg),None,non classé
4,CL,1337,CHLORURES,Chlorures,None,sous produit désinfection
5,CL,1337,CHLORURES,Chlorures,16887-00-6,sous produit désinfection
6,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),None,nitrite
7,NO2,1339,NITRITES (EN NO2),Nitrites (en NO2),14797-65-0,nitrite
8,NO3,1340,NITRATES (EN NO3),Nitrates (en NO3),None,nitrite
9,NO3,1340,NITRATES (EN NO3),Nitrates (en NO3),14797-55-8,nitrite


==> On a souvent deux lignes pour une même substance : avec et sans numéro CAS

In [16]:
# Compter le nombre de paramètre unique par cdparametre apres proposition de fix

query_casparam_doublons = """
WITH 
deduplicate_cas AS (
    SELECT
      cdparametre  ,
      MAX(casparam) AS casparam 
    FROM 
        mapping_categories
    GROUP BY
        cdparametre  
),
  
liste_doublon AS (
    SELECT
      cdparametre  ,
      COUNT(DISTINCT COALESCE(casparam,'NULL')	) AS nb_casparam 
    FROM 
        deduplicate_cas
    GROUP BY
        cdparametre  
    HAVING 
        nb_casparam >  1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametre = liste_doublon.cdparametre
ORDER BY 
    mapping_categories.cdparametre
"""

df_casparam_doublons = con.sql(query_casparam_doublons).df()
df_casparam_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie


In [17]:
query_Test2 = """
WITH 
ignore_cas AS (
    SELECT DISTINCT 
       * EXCLUDE (casparam)
    FROM 
        mapping_categories
)

SELECT
    cdparametresiseeaux,
    COUNT(*) AS nb
FROM 
    ignore_cas
GROUP BY
   1
HAVING 
    nb >1
"""
con.sql(query_Test2).show()

┌─────────────────────┬───────┐
│ cdparametresiseeaux │  nb   │
│       varchar       │ int64 │
├─────────────────────┼───────┤
│ IMAZAQU             │     2 │
│ PPCBS               │     2 │
│ SAXIT               │     2 │
│ CLTHALS             │     2 │
│ GIARDIA             │     2 │
│ MCTOT               │     2 │
│ MCTOAEL             │     2 │
│ URAN                │     2 │
│ ANTXA               │     2 │
│ PFOS                │     2 │
│ CYANO53             │     2 │
│ DSPHOS              │     2 │
│ MFTC                │     2 │
│ CYSPPS              │     2 │
├─────────────────────┴───────┤
│ 14 rows           2 columns │
└─────────────────────────────┘



In [18]:
query_Test2 = """
WITH 
ignore_cas AS (
    SELECT DISTINCT 
       * EXCLUDE (casparam)
    FROM 
        mapping_categories
)

SELECT
    cdparametresiseeaux,
    COUNT(*) AS nb
FROM 
    ignore_cas
GROUP BY
   1
HAVING 
    nb >1
"""
con.sql(query_Test2).show()

┌─────────────────────┬───────┐
│ cdparametresiseeaux │  nb   │
│       varchar       │ int64 │
├─────────────────────┼───────┤
│ URAN                │     2 │
│ ANTXA               │     2 │
│ CLTHALS             │     2 │
│ GIARDIA             │     2 │
│ MCTOAEL             │     2 │
│ MCTOT               │     2 │
│ IMAZAQU             │     2 │
│ PPCBS               │     2 │
│ SAXIT               │     2 │
│ MFTC                │     2 │
│ CYANO53             │     2 │
│ CYSPPS              │     2 │
│ DSPHOS              │     2 │
│ PFOS                │     2 │
├─────────────────────┴───────┤
│ 14 rows           2 columns │
└─────────────────────────────┘



In [19]:
query_casparam_doublons = """
WITH 
ignore_cas AS (
    SELECT DISTINCT 
       * EXCLUDE (casparam)
    FROM 
        mapping_categories
),
  
liste_doublon AS (
    SELECT
        cdparametresiseeaux,
        COUNT(*) AS nb
    FROM 
        ignore_cas
    GROUP BY
       1
    HAVING 
        nb >1
    )

SELECT
   mapping_categories.*
FROM 
     mapping_categories
INNER JOIN 
    liste_doublon
ON
    mapping_categories.cdparametresiseeaux = liste_doublon.cdparametresiseeaux
ORDER BY 
    mapping_categories.cdparametresiseeaux
"""

df_casparam_doublons = con.sql(query_casparam_doublons).df()
df_casparam_doublons

,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
0,ANTXA,7550,ANATOXINE A TOTALE,Anatoxine A totale,64285-06-9,microbio
1,ANTXA,7550,ANATOXINES A TOTALES,Anatoxines A totales,64285-06-9,microbio
2,CLTHALS,7717,CHLOROTHALONIL R417888,Chlorothalonil R417888,1418095-02-9,métabolite de pesticide
3,CLTHALS,7717,CHLOROTHALONIL SA,Chlorothalonil SA,1418095-02-9,métabolite de pesticide
4,CYANO53,<NA>,JAAGINEMA SP,None,None,microbio
5,CYANO53,<NA>,JAAGINEMA SP,Jaaginema sp,None,microbio
6,CYSPPS,7552,CYLINDROSPERMOPSINE TOTALE,Cylindrospermopsine totale,143545-90-8,microbio
7,CYSPPS,7552,CYLINDROSPERMOPSINES TOTALES,Cylindrospermopsines totales,143545-90-8,microbio
8,DSPHOS,7584,DISODIUM PHOSPHONATE,Disodium phosphonate,13708-85-5,non classé
9,DSPHOS,8511,ANION PHOSPHONATE,anion phosphonate,14901-63-4,non classé


# test SQL

In [2]:
# Compter le nombre de résultat

query_count_results = """
    SELECT
      COUNT(*)
    FROM 
        edc_resultats
"""

nb_result = con.sql(query_count_results).df()
nb_result

,count_star()
0,64164498


In [21]:
# Compter le nombre de résultat apres JOIN

query_count_results_join = """
    SELECT
      COUNT(*)
    FROM 
        edc_resultats
    LEFT JOIN
      mapping_categories
    ON 
      edc_resultats.cdparametre =   mapping_categories.cdparametre
      AND edc_resultats.libmajparametre = mapping_categories.libmajparametre
      AND  edc_resultats.cdparametresiseeaux = mapping_categories.cdparametresiseeaux
      AND  edc_resultats.casparam = mapping_categories.casparam
"""

nb_result_join = con.sql(query_count_results_join).df()
nb_result_join

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star()
0,64164498


In [22]:
if nb_result_join.iloc[0].values == nb_result.iloc[0].values:
    print("ok :)")
else:
    print("KO")

ok :)


# Proposition de table intermédiaire depuis mapping_categories

In [10]:
query_new_map = """
    SELECT
      cdparametresiseeaux  ,
      STRING_AGG( DISTINCT categorie) AS categorie,
      string_agg( DISTINCT cdparametre) AS cdparametre_list,
      string_agg( DISTINCT libmajparametre) AS libmajparametre_list,
      string_agg( DISTINCT libminparametre) AS libminparametre_list,
      string_agg( DISTINCT casparam) AS casparam_list,
    FROM 
      mapping_categories
    GROUP BY 
        cdparametresiseeaux
"""

new_map_df = con.sql(query_new_map).df()
new_map_df

,cdparametresiseeaux,categorie,cdparametre_list,libmajparametre_list,libminparametre_list,casparam_list
0,SCT,pesticides,1662,SULCOTRIONE,Sulcotrione,99105-77-8
1,OXDZ,pesticides,1667,OXADIAZON,Oxadiazon,19666-30-9
2,FIPRO,pesticides,2009,FIPRONIL,Fipronil,120068-37-3
3,DIMTH,pesticides,1175,DIMÉTHOATE,Diméthoate,60-51-5
4,FSLZ,pesticides,1194,FLUSILAZOL,Flusilazol,85509-19-9
...,...,...,...,...,...,...
1589,DIAZ,pesticides,1157,DIAZINON,Diazinon,333-41-5
1590,DILONE,pesticides,2983,DIFETHIALONE,Difethialone,104653-34-1
1591,ALKYL03,phénol,1957,NONYLPHÉNOL,Nonylphénol,25154-52-3
1592,DLL,pesticides,1156,DIALLATE,Diallate,2303-16-4


In [11]:
# Compter le nombre de résultat apres JOIN

query_count_results_join2 = """
    SELECT
      COUNT(*)
    FROM 
        edc_resultats
    LEFT JOIN
      new_map_df
    ON 
      edc_resultats.cdparametresiseeaux = new_map_df.cdparametresiseeaux
"""

nb_result_join2 = con.sql(query_count_results_join2).df()
nb_result_join2

,count_star()
0,64164498


In [12]:
if nb_result_join2.iloc[0].values == nb_result.iloc[0].values:
    print("ok :)")
else:
    print("KO")

ok :)


### ZOOM

In [14]:
query_new_map_ZOOM = """
    SELECT
      cdparametresiseeaux  ,
      STRING_AGG( DISTINCT categorie) AS categorie,
      string_agg( DISTINCT cdparametre) AS cdparametre_list,
      string_agg( DISTINCT libmajparametre) AS libmajparametre_list,
      string_agg( DISTINCT libminparametre) AS libminparametre_list,
      string_agg( DISTINCT casparam) AS casparam_list,
      COUNT(DISTINCT cdparametre) AS nb_cdparametre   ,
      COUNT(DISTINCT libmajparametre) AS nb_libmajparametre ,
      COUNT(DISTINCT libminparametre) AS nb_libminparametre ,	
      COUNT(DISTINCT casparam	) AS nb_casparam 
    FROM 
      mapping_categories
    GROUP BY 
        cdparametresiseeaux
    HAVING
        COUNT(DISTINCT categorie) != 1
"""

new_map_zoom_df = con.sql(query_new_map_ZOOM).df()
new_map_zoom_df

,cdparametresiseeaux,categorie,cdparametre_list,libmajparametre_list,libminparametre_list,casparam_list,nb_cdparametre,nb_libmajparametre,nb_libminparametre,nb_casparam
0,PFOS,"pfas,hydrocarbure",6561,"ACIDE SULFONIQUE DE PERFLUOROOCTANE,SULFONATE DE PERFLUOROOCTANE","Sulfonate de perfluorooctane,Acide sulfonique de perfluorooctane","1763-23-1,45298-90-6",1,2,2,2


In [15]:
query_new_map_ZOOM = """
    SELECT
      cdparametresiseeaux  ,
      STRING_AGG( DISTINCT categorie) AS categorie,
      string_agg( DISTINCT cdparametre) AS cdparametre_list,
      string_agg( DISTINCT libmajparametre) AS libmajparametre_list,
      string_agg( DISTINCT libminparametre) AS libminparametre_list,
      string_agg( DISTINCT casparam) AS casparam_list,
      COUNT(DISTINCT cdparametre) AS nb_cdparametre   ,
      COUNT(DISTINCT libmajparametre) AS nb_libmajparametre ,
      COUNT(DISTINCT libminparametre) AS nb_libminparametre ,	
      COUNT(DISTINCT casparam	) AS nb_casparam 
    FROM 
      mapping_categories
    GROUP BY 
        cdparametresiseeaux
    HAVING
       nb_cdparametre >1 
       OR nb_libmajparametre > 1
       OR nb_libminparametre >1
       OR nb_casparam > 1
    ORDER BY 
        nb_casparam DESC,
        nb_cdparametre DESC,
        nb_libmajparametre DESC
        
"""

new_map_zoom_df = con.sql(query_new_map_ZOOM).df()
new_map_zoom_df

,cdparametresiseeaux,categorie,cdparametre_list,libmajparametre_list,libminparametre_list,casparam_list,nb_cdparametre,nb_libmajparametre,nb_libminparametre,nb_casparam
0,DSPHOS,non classé,"7584,8511","DISODIUM PHOSPHONATE,ANION PHOSPHONATE","anion phosphonate,Disodium phosphonate","14901-63-4,13708-85-5",2,2,2,2
1,PFOS,"hydrocarbure,pfas",6561,"ACIDE SULFONIQUE DE PERFLUOROOCTANE,SULFONATE DE PERFLUOROOCTANE","Acide sulfonique de perfluorooctane,Sulfonate de perfluorooctane","1763-23-1,45298-90-6",1,2,2,2
2,URAN,non classé,1361,"URANIUM EN MICROGRAMME PAR LITRE,URANIUM EN ΜG/L",Uranium en µg/l,7440-61-1,1,2,1,1
3,ANTXA,microbio,7550,"ANATOXINE A TOTALE,ANATOXINES A TOTALES","Anatoxines A totales,Anatoxine A totale",64285-06-9,1,2,2,1
4,CYSPPS,microbio,7552,"CYLINDROSPERMOPSINES TOTALES,CYLINDROSPERMOPSINE TOTALE","Cylindrospermopsine totale,Cylindrospermopsines totales",143545-90-8,1,2,2,1
5,IMAZAQU,pesticides,2860,"IMIZAQUINE,IMAZAQUINE","Imizaquine,Imazaquine",81335-37-7,1,2,2,1
6,SAXIT,microbio,7551,"SAXITOXINES TOTALES,SAXITOXINE TOTALE","Saxitoxine totale,Saxitoxines totales",35523-89-8,1,2,2,1
7,PPCBS,pesticides,5602,"PROPOXYCARBAZONE,PROPOXYCARBAZONE-SODIUM","Propoxycarbazone-sodium,Propoxycarbazone",181274-15-7,1,2,2,1
8,CLTHALS,métabolite de pesticide,7717,"CHLOROTHALONIL SA,CHLOROTHALONIL R417888","Chlorothalonil R417888,Chlorothalonil SA",1418095-02-9,1,2,2,1
9,MCTOAEL,microbio,6278,"TOTAL DES MICROCYSTINES ANALYSÉES - TEST ELISA,MICROCYSTINES ANALYSÉES -TEST ELISA","Total des microcystines analysées - test ELISA,Microcystines analysées -test ELISA",None,1,2,2,0
